In [17]:
import numpy as np
import pandas as pd
import snap
import matplotlib.pyplot as plt

In [18]:
#Load edge list into TTable
context = snap.TTableContext()
edgefilename = "./data/InvInvPairs.csv"
nodefilename = "./data/Investor.csv"
edgeschema = snap.Schema()
edgeschema.Add(snap.TStrTAttrPr("srcNID", snap.atInt))
edgeschema.Add(snap.TStrTAttrPr("dstNID", snap.atInt))
edgeschema.Add(snap.TStrTAttrPr("weight", snap.atFlt))

nodeschema = snap.Schema()
nodeschema.Add(snap.TStrTAttrPr("investorID", snap.atStr))
nodeschema.Add(snap.TStrTAttrPr("investorname", snap.atStr))

edge_table = snap.TTable.LoadSS(edgeschema, edgefilename, context, "\t", snap.TBool(False))
node_table = snap.TTable.LoadSS(nodeschema, nodefilename, context, "\t", snap.TBool(False))

In [19]:
#Convert TTable into Directed graph
edgeattrv = snap.TStrV()
edgeattrv.Add("weight")

nodeattrv = snap.TStrV()
nodeattrv.Add("investorname")
net = snap.ToNetwork(snap.PNEANet, edge_table, "srcNID", "dstNID", edgeattrv, node_table, "investorID", nodeattrv, snap.aaFirst)

In [20]:
NIdHubH = snap.TIntFltH()
NIdAuthH = snap.TIntFltH()
snap.GetHits(net, NIdHubH, NIdAuthH)
Hubs = {}
for item in NIdHubH:
    Hubs[item] = NIdHubH[item]
sorted_Hubs = sorted(Hubs.items(), key = lambda kv: -kv[1])

Auths = {}
for item in NIdAuthH:
    Auths[item] = NIdAuthH[item]
sorted_Auths = sorted(Auths.items(), key = lambda kv: -kv[1])

NameV = snap.TStrV()

df_investors = pd.read_csv('./data/Investor.csv', header=None, sep = '\t')
df_investors = df_investors.rename(columns = {0:'investorID', 1:'investorname'})

print("Hubbiness")
for i in range(10):
    NId = sorted_Hubs[i][0]
    score = sorted_Hubs[i][1]
    net.AttrNameNI(NId, NameV)
    investorID = 0
    investorname = df_investors[df_investors['investorID']==NId].investorname.values[0] 
    indeg = net.GetNI(NId).GetInDeg()
    outdeg = net.GetNI(NId).GetOutDeg()
    print(' nodeID=',NId, ' hubiness=',score, ' name=',investorname, ' indeg=', indeg, ' outdeg=',outdeg)

print("Authority")
for i in range(10):
    NId = sorted_Auths[i][0]
    score = sorted_Auths[i][1]
    investorname = df_investors[df_investors['investorID']==NId].investorname.values[0]
    indeg = net.GetNI(NId).GetInDeg()
    outdeg = net.GetNI(NId).GetOutDeg()
    print(' nodeID=',NId, ' authority=',score, ' name=',investorname, ' indeg=', indeg, ' outdeg=',outdeg)

Hubbiness
 nodeID= 1824  hubiness= 0.17665723158963262  name= FAMILY LEGACY INC  indeg= 15  outdeg= 191
 nodeID= 233  hubiness= 0.12929754556009204  name= AMERICAN ECONOMIC PLANNING GROUP INC  indeg= 15  outdeg= 135
 nodeID= 3166  hubiness= 0.10560340178884439  name= M KRAUS CO  indeg= 15  outdeg= 114
 nodeID= 1229  hubiness= 0.10340095195832286  name= COHO PARTNERS LTD  indeg= 15  outdeg= 119
 nodeID= 5021  hubiness= 0.10137017732441407  name= SUMMIT ASSET MANAGEMENT LLC  indeg= 15  outdeg= 113
 nodeID= 1418  hubiness= 0.10134848397114343  name= CTC LLC  indeg= 15  outdeg= 118
 nodeID= 1919  hubiness= 0.09417869921989513  name= FIRST LONG ISLAND INVESTORS LLC  indeg= 15  outdeg= 115
 nodeID= 2046  hubiness= 0.08660459692920182  name= FRONT BARNETT ASSOCIATES LLC  indeg= 15  outdeg= 108
 nodeID= 1800  hubiness= 0.08428706435855499  name= EXCALIBUR MANAGEMENT CORP  indeg= 15  outdeg= 102
 nodeID= 854  hubiness= 0.08398584339376869  name= BROUWER JANACHOWSKI LLC  indeg= 15  outdeg= 99
Au

In [21]:
#Hubbiness and Authority of the value investors 
valueinvestor = pd.read_csv('./data/valueinvestorname.txt', names = {'investorname'})
valueinvestor = valueinvestor.append(df_investors[df_investors['investorname'].str.contains('VALUE')][['investorname']], ignore_index = True)
valueinvestor = valueinvestor.merge(df_investors, on = ['investorname'], how = 'left')
valueinvestor = valueinvestor[valueinvestor.investorID.notnull()]
valueinvestor = valueinvestor.astype({'investorID':'uint16'})
auths = pd.DataFrame(sorted_Auths, columns = ['investorID', 'Auth'])
valueinvestor = valueinvestor.merge(auths, on = ['investorID'], how = 'left')
hubs = pd.DataFrame(sorted_Hubs, columns = ['investorID', 'Hub'])
valueinvestor = valueinvestor.merge(hubs, on = ['investorID'], how = 'left')
valueinvestor.sort_values(by = ['Hub'], ascending=False)

,investorname,investorID,Auth,Hub
35,COHO PARTNERS LTD,1229,0.011833,0.103401
40,CHEVIOT VALUE MANAGEMENT LLC,1118,0.024261,0.081488
25,RIVERPARK ADVISORS LLC,4374,0.011752,0.041307
0,BERKSHIRE HATHAWAY INC,648,0.014643,0.033161
37,PERSHING SQUARE CAPITAL MANAGEMENT LP,4003,0.010425,0.031092
64,VALUEACT HOLDINGS LP,5427,0.014307,0.028004
26,TWEEDY BROWNE CO LLC 1,5356,0.015993,0.027851
66,VALUEWORKS LLC,5429,0.011485,0.026243
10,VALUEWORKS LLC,5429,0.011485,0.026243
41,DECCAN VALUE INVESTORS LP,1500,0.021963,0.019679


In [26]:
hits = auths.merge(hubs, on = ['investorID'], how='left')
hits.to_csv("./data/hits.csv", header=True, index=False, sep="\t")